In [1]:
# main libraries
import os
import json
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from groq import Groq

# typing and classes
import instructor
from pydantic import BaseModel, Field
from typing import Optional, Literal, List
from enum import Enum 

# constants
load_dotenv("../.env")
GROQ_KEY = os.getenv("GROQ_KEY")
MODEL = "llama-3.3-70b-versatile"

# initialisation of model
client = Groq(api_key=GROQ_KEY)
client = instructor.from_groq(client, mode=instructor.Mode.JSON)
print("Groq client initialised.")

Groq client initialised.


In [2]:
'''
Enumerations needed:
- CPU (microarchitectures, integrated_graphics)
- Cooler (color)
- Internal Hard Drive (type, form_factor, interface)
- Memory (color)
- Motherboard (cpu_socket, form_factor, colour)
'''

def create_enum_from_list(enum_name: str, values: list):
    enum_dict = {v.lower().replace(" ", "_").replace("+", "plus").replace("-", "_"): v for v in values}
    return Enum(enum_name, enum_dict)

cpu_df = pd.read_csv("../../data/cpu.csv")
cooler_df = pd.read_csv("../../data/cooler.csv")
storage_df = pd.read_csv("../../data/storage.csv")
memory_df = pd.read_csv("../../data/memory.csv")
motherboard_df = pd.read_csv("../../data/motherboard.csv")

CPU_MICROARCHITECTURES = sorted(cpu_df['microarchitecture'].dropna().unique().tolist())
CPU_GRAPHICS = sorted(cpu_df['integrated_graphics'].dropna().unique().tolist())
COOLER_COLOURS = sorted(cooler_df['color'].dropna().unique().tolist())
STORAGE_TYPES = sorted(storage_df['type'].dropna().unique().tolist())
STORAGE_FORM_FACTORS = sorted(storage_df['form_factor'].dropna().unique().tolist())
STORAGE_INTERFACES = sorted(storage_df['interface'].dropna().unique().tolist())
MEMORY_COLOURS = sorted(memory_df['color'].dropna().unique().tolist())
MOTHERBOARD_SOCKETS = sorted(motherboard_df['cpu_socket'].dropna().unique().tolist())
MOTHERBOARD_FORM_FACTORS = sorted(motherboard_df['form_factor'].dropna().unique().tolist())
MOTHERBOARD_COLOURS = sorted(motherboard_df['color'].dropna().unique().tolist())

# further processing
def get_colours(colour_array: List) -> List:
    l = [list(map(str.strip, entry.split('/'))) for entry in colour_array]
    l = list(set(np.hstack([x for x in l])))
    return l

COOLER_COLOURS = get_colours(COOLER_COLOURS)
MEMORY_COLOURS = get_colours(MEMORY_COLOURS)
MOTHERBOARD_COLOURS = get_colours(MOTHERBOARD_COLOURS)

Microarchitecture = create_enum_from_list("microarchitecture", CPU_MICROARCHITECTURES)
IntegratedGraphics = create_enum_from_list("IntegratedGraphics", CPU_GRAPHICS)
CoolerColour = create_enum_from_list("cooler_colour", COOLER_COLOURS)
StorageType = create_enum_from_list("storage_type", STORAGE_TYPES)
StorageFormFactor = create_enum_from_list("storage_form_factor", STORAGE_FORM_FACTORS)
StorageInterface = create_enum_from_list("storage_interface", STORAGE_INTERFACES)
MemoryColour = create_enum_from_list("memory_colour", MEMORY_COLOURS)
MotherboardSocket = create_enum_from_list("motherboard_socket", MOTHERBOARD_SOCKETS)
MotherboardFormFactor = create_enum_from_list("motherboard_form_factor", MOTHERBOARD_FORM_FACTORS)
MotherboardColour = create_enum_from_list("motherboard_clour", MOTHERBOARD_COLOURS)

In [3]:
class CPURequirements(BaseModel):
    min_cores: Optional[int] = Field(None, description="Minimum number of CPU cores desired")
    min_core_clock_ghz: Optional[float] = Field(None, description="Minimum core clock speed (in GHz)")
    min_boost_clock_ghz: Optional[float] = Field(None, description="Minimum boost clock speed (in GHz)")
    microarchitecture: Optional[List[Microarchitecture]] = Field(None, description="Preferred CPU microarchitecture, e.g., ['Zen 4', 'Raptor Lake'])")
    max_tdp_watts: Optional[int] = Field(None, description="Maximum thermal design power (in watts)")
    #needs_integrated_graphics: Optional[bool] = Field(None, description="Whether integrated graphics are required")
    #min_rating: Optional[float] = Field(None, description="Minimum user rating (out of 5)")
    max_price: Optional[float] = Field(None, description="Maximum budget for CPU (in USD)")

class CoolerRequirements(BaseModel):
    min_fan_rpm: Optional[int] = Field(None, description="Minimum fan RPM for cooling performance")
    max_noise_level_db: Optional[float] = Field(None, description="Desired maximum noise level in decibels")
    max_radiator_size_mm: Optional[int] = Field(None, description="Maximum radiator size in mm (e.g. 240, 360)")
    #preferred_color: Optional[List[CoolerColour]] = Field(None, description="Preferred cooler colors")
    #min_rating: Optional[float] = Field(None, description="Minimum user rating (out of 5)")
    max_price: Optional[float] = Field(None, description="Maximum budget for cooler (in USD)")

class StorageRequirements(BaseModel):
    min_capacity_gb: Optional[int] = Field(None, description="Minimum storage capacity in GB")
    preferred_type: Optional[List[StorageType]] = Field(None, description="Drive type (e.g. ['SSD', 'HDD'])")
    min_cache_gb: Optional[float] = Field(None, description="Minimum cache size in GB (e.g. 2.048, 0.512) (this should be a small number)")
    preferred_form_factor: Optional[List[StorageFormFactor]] = Field(None, description="Preferred drive form factor (e.g. ['2.5', 'M.2'])")
    preferred_interface: Optional[List[StorageInterface]] = Field(None, description="Preferred interface (e.g. ['SATA, NVMe'])")
    #min_rating: Optional[float] = Field(None, description="Minimum user rating (out of 5)")
    max_price_per_gb: Optional[float] = Field(None, description="Maximum budget for drive for each gigabyte (in USD)")

class MemoryRequirements(BaseModel):
    min_capacity_gb: Optional[int] = Field(None, description="Minimum total memory capacity in GB")
    min_speed_mhz: Optional[int] = Field(None, description="Minimum memory speed in MHz")
    preferred_module_count: Optional[int] = Field(None, description="Preferred number of modules (e.g. 2 for dual-channel)")
    max_cas_latency: Optional[float] = Field(None, description="Maximum acceptable CAS latency")
    #max_first_word_latency: Optional[float] = Field(None, description="Maximum acceptable first-word latency (in ns)")
    #preferred_color: Optional[List[MemoryColour]] = Field(None, description="Preferred memory colors")
    #min_rating: Optional[float] = Field(None, description="Minimum user rating (out of 5)")
    max_price: Optional[float] = Field(None, description="Maximum budget for memory (in USD)")

class MotherboardRequirements(BaseModel):
    preferred_socket: Optional[List[MotherboardSocket]] = Field(None, description="CPU socket type required (e.g. AM5, LGA1700)")
    preferred_form_factor: Optional[List[MotherboardFormFactor]] = Field(None, description="Motherboard form factor (e.g. ATX, Micro ATX)")
    min_max_memory_gb: Optional[int] = Field(None, description="Minimum supported max memory in GB")
    min_memory_slots: Optional[int] = Field(None, description="Minimum number of memory slots")
    #preferred_color: Optional[List[MotherboardColour]] = Field(None, description="Preferred motherboard color")
    #min_rating: Optional[float] = Field(None, description="Minimum user rating (out of 5)")
    max_price: Optional[float] = Field(None, description="Maximum budget for case (in USD)")

class PCRequirements(BaseModel):
    cpu: CPURequirements = Field(None, description="Details of preferred CPU")
    cooler: CoolerRequirements = Field(None, description="Details of preferred CPU cooler")
    storage: StorageRequirements = Field(None, description="Details of preferred internal hard drive for storage")
    memory: MemoryRequirements = Field(None, description="Details of preferred memory RAM")
    motherboard: MotherboardRequirements = Field(None, description="Details of preferred motherboard")

In [4]:
message = "I want a decent PC rig that does things fast and has a lot of RAM!"

cpu_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=CPURequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

cooler_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=CoolerRequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

storage_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=StorageRequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

memory_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=MemoryRequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

motherboard_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=MotherboardRequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

In [5]:
cpu_requirements

{'min_cores': 8,
 'min_core_clock_ghz': 3.5,
 'min_boost_clock_ghz': 4.5,
 'microarchitecture': [<microarchitecture.zen_4: 'Zen 4'>,
  <microarchitecture.raptor_lake: 'Raptor Lake'>],
 'max_tdp_watts': 170,
 'max_price': 800.0}

In [6]:
cooler_requirements

{'min_fan_rpm': 1000,
 'max_noise_level_db': 40.0,
 'max_radiator_size_mm': 360,
 'max_price': 150.0}

In [7]:
storage_requirements

{'min_capacity_gb': 1024,
 'preferred_type': [<storage_type.ssd: 'SSD'>],
 'min_cache_gb': 2.0,
 'preferred_form_factor': [<storage_form_factor.m.2_2280: 'M.2-2280'>],
 'preferred_interface': [<storage_interface.m.2_pcie_4.0_x4: 'M.2 PCIe 4.0 X4'>],
 'max_price_per_gb': 0.1}

In [8]:
memory_requirements

{'min_capacity_gb': 16,
 'min_speed_mhz': 3200,
 'preferred_module_count': 2,
 'max_cas_latency': 18.0,
 'max_price': 200.0}

In [9]:
motherboard_requirements

{'preferred_socket': [<motherboard_socket.lga1700: 'LGA1700'>,
  <motherboard_socket.am5: 'AM5'>],
 'preferred_form_factor': [<motherboard_form_factor.atx: 'ATX'>,
  <motherboard_form_factor.micro_atx: 'Micro ATX'>],
 'min_max_memory_gb': 128,
 'min_memory_slots': 4,
 'max_price': 800.0}

In [10]:
pc_requirements = client.chat.completions.create(
    model=MODEL,
    response_model=PCRequirements,
    messages=[
        {"role": "user", "content": message}
    ]
).model_dump()

pc_requirements

{'cpu': {'min_cores': 8,
  'min_core_clock_ghz': 3.5,
  'min_boost_clock_ghz': 4.5,
  'microarchitecture': [<microarchitecture.zen_4: 'Zen 4'>,
   <microarchitecture.raptor_lake: 'Raptor Lake'>],
  'max_tdp_watts': 125,
  'max_price': 500.0},
 'cooler': {'min_fan_rpm': 1000,
  'max_noise_level_db': 30.0,
  'max_radiator_size_mm': 360,
  'max_price': 100.0},
 'storage': {'min_capacity_gb': 1000,
  'preferred_type': [<storage_type.ssd: 'SSD'>],
  'min_cache_gb': 2.048,
  'preferred_form_factor': [<storage_form_factor.m.2_2280: 'M.2-2280'>],
  'preferred_interface': [<storage_interface.m.2_pcie_4.0_x4: 'M.2 PCIe 4.0 X4'>],
  'max_price_per_gb': 0.1},
 'memory': {'min_capacity_gb': 32,
  'min_speed_mhz': 3200,
  'preferred_module_count': 4,
  'max_cas_latency': 16.0,
  'max_price': 200.0},
 'motherboard': {'preferred_socket': [<motherboard_socket.am5: 'AM5'>,
   <motherboard_socket.lga1700: 'LGA1700'>],
  'preferred_form_factor': [<motherboard_form_factor.atx: 'ATX'>],
  'min_max_memory_gb

In [11]:
[pc_requirements[x] for x in pc_requirements.keys()]

[{'min_cores': 8,
  'min_core_clock_ghz': 3.5,
  'min_boost_clock_ghz': 4.5,
  'microarchitecture': [<microarchitecture.zen_4: 'Zen 4'>,
   <microarchitecture.raptor_lake: 'Raptor Lake'>],
  'max_tdp_watts': 125,
  'max_price': 500.0},
 {'min_fan_rpm': 1000,
  'max_noise_level_db': 30.0,
  'max_radiator_size_mm': 360,
  'max_price': 100.0},
 {'min_capacity_gb': 1000,
  'preferred_type': [<storage_type.ssd: 'SSD'>],
  'min_cache_gb': 2.048,
  'preferred_form_factor': [<storage_form_factor.m.2_2280: 'M.2-2280'>],
  'preferred_interface': [<storage_interface.m.2_pcie_4.0_x4: 'M.2 PCIe 4.0 X4'>],
  'max_price_per_gb': 0.1},
 {'min_capacity_gb': 32,
  'min_speed_mhz': 3200,
  'preferred_module_count': 4,
  'max_cas_latency': 16.0,
  'max_price': 200.0},
 {'preferred_socket': [<motherboard_socket.am5: 'AM5'>,
   <motherboard_socket.lga1700: 'LGA1700'>],
  'preferred_form_factor': [<motherboard_form_factor.atx: 'ATX'>],
  'min_max_memory_gb': 128,
  'min_memory_slots': 4,
  'max_price': 300.0

In [12]:
with open('./requirements/cpu_requirements.json', 'w') as f:
    json.dump(cpu_requirements, f, default=lambda x: x._name_)
with open('./requirements/cooler_requirements.json', 'w') as f:
    json.dump(cooler_requirements, f, default=lambda x: x._name_)
with open('./requirements/storage_requirements.json', 'w') as f:
    json.dump(storage_requirements, f, default=lambda x: x._name_)
with open('./requirements/memory_requirements.json', 'w') as f:
    json.dump(memory_requirements, f, default=lambda x: x._name_)
with open('./requirements/motherboard_requirements.json', 'w') as f:
    json.dump(motherboard_requirements, f, default=lambda x: x._name_)

FileNotFoundError: [Errno 2] No such file or directory: './requirements/cpu_requirements.json'